# Domain Definitions

In [315]:
import pandas as pd
from dataclasses import dataclass
from numpy.random import default_rng
from numpy.random import Generator
from pathlib import Path
from typing import Optional, List, Set, Dict, Tuple

In [316]:
@dataclass(frozen=True)
class Item:
    name: str
    value: float
    weight: int
    time_to_steal: int

    @property
    def tts(self):
        return self.time_to_steal

In [362]:
@dataclass(frozen=False)
class Node:
    name: str
    item: Item

    def __init__(self, name: str):
        self.name = name
        self.item = None

    def with_item(self, name: str, value: float, weight, tts: int) -> 'Node':
        return self._with_item(Item(name, value, weight, time_to_steal=tts))

    def _with_item(self, item: Item) -> 'Node':
        self.item = item
        return self

    def to_dict(self):
        try:
            return {
                'name': self.name,
                'item': self.item.name,
                'value': self.item.value,
                'weight': self.item.weight,
                'tts': self.item.tts
            }
        except Exception:
            return {
                'name': self.name,
                'item': None,
            }

    def __hash__(self) -> int:
        return self.name.__hash__()

In [363]:
@dataclass(frozen=True)
class Edge:
    source: Node
    destination: Node
    travelling_time: int
    travelling_cost: float

    @property
    def time(self):
        return self.travelling_time

    @property
    def cost(self):
        return self.travelling_cost

    def to_dict(self):
        return {
            'source': self.source.name,
            'destination': self.destination.name,
            'travelling_time': self.time,
            'travelling_cost': self.cost
        }

In [364]:
class Graph:
    nodes: Dict[str, Node]
    edges: Dict[Tuple[Node, Node], Edge]
    connections: Dict[Node, Set[Edge]]

    def __init__(self):
        self.nodes = dict()
        self.edges = dict()
        self.connections = dict()

    def get_node(self, name: str) -> Optional[Node]:
        return self.nodes.get(name)

    def get_all_nodes(self) -> List[Node]:
        return list(self.nodes.values())

    def get_all_edges(self) -> List[Edge]:
        return list(self.edges.values())

    def get_connections_from_node(self, node: Node) -> List[Edge]:
        return self.connections.get(node, {})

    def get_connections_from(self, node_name: str) -> List[Edge]:
        node: Optional[Node] = self.get_node(node_name)
        return self.get_connections_from_node(node)

    def find_connection_between(self, src: Node, dst: Node) -> Optional[Edge]:
        return self.edges.get((src, dst), None)

    def add_edge(self, src: str, dst: str, tt: int, tc: float) -> 'Graph':
        source = self.nodes.get(src, Node(src))
        destination = self.nodes.get(dst, Node(dst))
        edge = Edge(source, destination, travelling_time=tt, travelling_cost=tc)
        self._add_node(source)._add_node(destination)
        self.edges[(source, destination)] = edge
        self.connections[source] = self.connections.get(source, []) + [edge]
        return self

    def add_node(self, node_name: str) -> 'Graph':
        self.nodes[node_name] = Node(node_name)
        return self

    def _add_node(self, node: Node) -> 'Graph':
        self.nodes[node.name] = node
        return self

# Graph Construction

In [365]:
datasets_dir = Path().absolute().parent.joinpath('datasets')
travelling_map = f"{datasets_dir}/travelling_map.csv"
treasure_map = f"{datasets_dir}/treasure_map.csv"

In [366]:
travels = pd.read_csv(travelling_map)
treasures = pd.read_csv(treasure_map)

In [367]:
g = Graph()

In [368]:
travels.apply(lambda row: g.add_edge(src=row['source'],
                                     dst=row['destination'],
                                     tt=row['travelling_time'],
                                     tc=row['travelling_cost'])
                           .add_edge(src=row['destination'],
                                     dst=row['source'],
                                     tt=row['travelling_time'],
                                     tc=row['travelling_cost']), axis=1)

treasures.apply(lambda row: g.get_node(row['city'])
                             .with_item(row['name'],
                                        row['value'],
                                        row['weight'],
                                        row['tts']), axis=1);

# Problem Solving 

In [466]:
@dataclass(frozen=False)
class Individual:
    genes: List[Node]
    
    total_income: float
    travelling_cost: float
    traveling_time: int
    time_to_steal: int
    carry_weight: int

    @property
    def profit(self):
        return self.total_income - self.travelling_cost

    @property
    def total_time(self):
        return self.traveling_time + self.time_to_steal

    @property
    def score(self):
        return self.profit/self.total_time

In [547]:
class GeneticAlgorithm:
    graph: Graph
    genes: Set[Node]
    max_hours: int
    max_weight: int
    rng: Generator

    def __init__(self, graph: Graph, with_max_hours: int, with_max_weight: int):
        self.graph = graph
        self.genes = list(graph.get_all_nodes())
        self.max_hours = with_max_hours
        self.max_weight = with_max_weight
        self.rng = default_rng()

    def generate_initial_batch(self, size: int = 10) -> List[Individual]:
        length = len(self.genes)
        return [self.generate_individual(length) for i in range(size)]

    def generate_individual(self, number_of_genes: int):
        while True:
            genes = self.generate_individual_genes(number_of_genes)
            tuples: Tuple[Node, Node] = [(genes[idx], genes[num]) for idx, num in enumerate(range(1, len(genes)))]
            connections: List[Edge] = [self.graph.find_connection_between(src, dest) for src, dest in tuples if (src != dest)]

            if len(connections) == 0:
                continue

            travelling_cost: float = 0.0
            travelling_time: int = 0
            total_income: float = 0.0
            time_to_steal: int = 0
            total_weight: int = 0

            for cx in connections:
                travelling_cost += cx.travelling_cost
                travelling_time += cx.travelling_time
                total_income += cx.destination.to_dict().get('value', 0)
                time_to_steal += cx.destination.to_dict().get('tts', 0)
                total_weight += cx.destination.to_dict().get('weight', 0)

            indv = Individual(genes,
                              total_income=total_income,
                              time_to_steal=time_to_steal,
                              travelling_cost=travelling_cost,
                              traveling_time=travelling_time,
                              carry_weight=total_weight)

            if self.fitness(indv) > 0:
                return indv

    def generate_individual_genes(self, number: int) -> List[Node]:
        indexes = self.rng.choice(number, size=number, replace=False)
        genes: List[Node] = [self.genes[idx] for idx in indexes]

        starting_point: Node = self.graph.get_node("Escondidos")
        endpoint_idx: int = (genes.index(starting_point))
        individual_genes: List[Node] = [starting_point] + genes[0:endpoint_idx+1]
        return individual_genes

    def fitness(self, indv: Individual) -> int:
        if indv.carry_weight > self.max_weight or indv.total_time > self.max_hours:
            return 0

        return indv.score

    def mutation(self):
        pass


    def cross_over(self):
        pass

In [548]:
ga = GeneticAlgorithm(g, with_max_hours=72, with_max_weight=20)

In [549]:
individuals: List[Individual] = ga.generate_initial_batch(size=10)

In [551]:
len(individuals)

10

In [552]:
for i in individuals:
    print(i.score)

329.8181818181818
319.81159420289856
360.4492753623188
403.81944444444446
189.55882352941177
307.1764705882353
336.390243902439
80.0
324.0
216.5


In [563]:
x = individuals[2]
y = individuals[1]

In [564]:
x.genes[1:-1]

[Node(name='Campos', item=Item(name='Espada sagrada', value=6500, weight=6, time_to_steal=5)),
 Node(name='Lagos', item=Item(name='Primeira moeda de $1 do país', value=3000, weight=1, time_to_steal=7)),
 Node(name='Leão', item=Item(name='Taça da Copa do Mundo de Corrida de Cavalo', value=4000, weight=2, time_to_steal=5)),
 Node(name='Além-do-Mar', item=Item(name='Maior diamante do continente', value=5400, weight=2, time_to_steal=10)),
 Node(name='Porto', item=Item(name='Fecha do caçador pré-histórico', value=2300, weight=1, time_to_steal=2)),
 Node(name='Guardião', item=Item(name='Primeira edição do livro "O Livro Azul"', value=3000, weight=1, time_to_steal=2)),
 Node(name='Limões', item=Item(name='Capacete de guerra antigo', value=4000, weight=2, time_to_steal=4))]

In [565]:
y.genes[1:-1]

[Node(name='Guardião', item=Item(name='Primeira edição do livro "O Livro Azul"', value=3000, weight=1, time_to_steal=2)),
 Node(name='Ponte-do-Sol', item=Item(name='Busto do líder da Revolução Pavão', value=1500, weight=6, time_to_steal=5)),
 Node(name='Além-do-Mar', item=Item(name='Maior diamante do continente', value=5400, weight=2, time_to_steal=10)),
 Node(name='Campos', item=Item(name='Espada sagrada', value=6500, weight=6, time_to_steal=5)),
 Node(name='Leão', item=Item(name='Taça da Copa do Mundo de Corrida de Cavalo', value=4000, weight=2, time_to_steal=5)),
 Node(name='Algas', item=Item(name='Colar de casamento da rainha Vanessa', value=2500, weight=1, time_to_steal=7)),
 Node(name='Porto', item=Item(name='Fecha do caçador pré-histórico', value=2300, weight=1, time_to_steal=2))]